# Linear Classification

In this lab you will implement parts of a linear classification model using the regularized empirical risk minimization principle. By completing this lab and analysing the code, you gain deeper understanding of these type of models, and of gradient descent.


## Problem Setting

The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images. Each of the patients is classified into two categories: normal (1) and abnormal (0). The training data contains 80 SPECT images from which 22 binary features have been extracted. The goal is to predict the label for an unseen test set of 187 tomography images.

In [1]:
import urllib.request
import pandas as pd
import numpy as np
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

testfile = urllib.request.URLopener()
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.train", "SPECT.train")
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.test", "SPECT.test")

df_train = pd.read_csv('SPECT.train',header=None)
df_test = pd.read_csv('SPECT.test',header=None)

train = df_train.values
test = df_test.values

y_train = train[:,0]
X_train = train[:,1:]
y_test = test[:,0]
X_test = test[:,1:]

### Exercise 1

Analyze the function learn_reg_ERM(X,y,lambda) which for a given $n\times m$ data matrix $\textbf{X}$ and binary class label $\textbf{y}$ learns and returns a linear model $\textbf{w}$.
The binary class label has to be transformed so that its range is $\left \{-1,1 \right \}$. 
The trade-off parameter between the empirical loss and the regularizer is given by $\lambda > 0$. 
To adapt the learning rate the Barzilai-Borwein method is used.

Try to understand each step of the learning algorithm and comment each line.


In [20]:
def learn_reg_ERM(X,y,lbda):
    #specify how many iterations algorithm/loop should run
    max_iter = 200
    
    #limit: if the weight is lower than the value e the loop will be stopped
    e  = 0.001
    
    #learning rate
    alpha = 1.

    # initialising weight vector with random values     
    w = np.random.randn(X.shape[1]);
    
    
    for k in np.arange(max_iter):
        # dot product of the input data and the weight
        h = np.dot(X,w)
        
        # calculating the loss by comparing the calculated dot product and real value y 
        l,lg = loss(h, y)
        
        # printing the current loss in a clear format by calulcating it's mean across all values
        print ('loss: {}'.format(np.mean(l)))
        
        # the function reg should ultimately compute the regulisation term (r) 
        # as well as it's gradient (rg - de: Ableitung) 
        r,rg = reg(w, lbda)
        
        # calculate the gradient of the loss function itself
        # this gradient will then point us in the direction of the loss functions steepest descent (Minima) ?
        g = np.dot(X.T,lg) + rg 
        
        # we want the loss function to be minimal
        # calculated by adding the dot product of the transposed input vector (X) 
        # and the gradient of the loss term with the gradient of the regulisation term
        if (k > 0):
            
            # calulate the new step size for the following caluclation of w
            # terms and values: 
                # alpha = learning rate - step size for updating our model
                # g_old.T = previous gradients transposed
                # g_old = previous gradients
                # g = gradient of the loss function
            # we change the alpha based on how much the gradient chaged 
            alpha = alpha * (np.dot(g_old.T,g_old))/(np.dot((g_old - g).T,g_old)) 
            # updated to make model more stable 
        
        # update the weight vector using our new gradient and newly calulated alpha
        # points in direction of steepest descent due to '-'
        w = w - alpha * g
        
        # check the change in weight and compare it to limit e
        # check if the iteration should be stopped (happens if improvement very small)
        if (np.linalg.norm(alpha * g) < e):
            break
            
        # save current gradient from this iteration to work with it in the next iteration
        g_old = g
    
    # return weight vector
    return w

### Exercise 2

Fill in the code for the function loss(h,y) which computes the hinge loss and its gradient. 
This function takes a given vector $\textbf{y}$ with the true labels $\in \left \{-1,1\right \}$ and a vector $\textbf{h}$ with the function values of the linear model as inputs. The function returns a vector $\textbf{l}$ with the hinge loss $\max(0, 1 − y_{i} h_{i})$ and a vector $\textbf{g}$ with the gradients of the hinge loss w.r.t $\textbf{h}$. (Note: The partial derivative of the hinge loss with respect to $\textbf{h}$  is $g_{i} = −y $ if $l_{i} > 0$, else $g_{i} = 0$)

In [7]:
def loss(h, y):
    l = []
    g = []
    for hi, yi in zip(h,y):
        l_current = max(0, 1-hi*yi) #hi = fTeta(xi)
        l.append (l_current)
        if l_current > 0:
            g.append(-yi)
        else:
            g.append(e)
    return l, g

In [13]:
def loss(h, y) :
    l: np.array = np.maximum(0, 1 - h*y)
    g: np.array = np.zeros(h. shape)
    g[np.where(1 > 0)] = y[np.where(1 > 0)] * -1
    return l, g

In [21]:
def loss(h, y):
    # compute hinge loss 
    l = np.maximum(0, 1 - h * y)
    
    # compute gradient of hinge loss
    g = np.where(1 > 0, -y, 0)
    
    return l, g

### Exercise 3

Fill in the code for the function reg(w,lambda) which computes the $\mathcal{L}_2$-regularizer and the gradient of the regularizer function at point $\textbf{w}$. 


$$r = \frac{\lambda}{2} \textbf{w}^{T}\textbf{w}$$

$$g = \lambda \textbf{w}$$

In [22]:
def reg(w, lbda):
    # compute l2-regulariser 
    r: np.array = (lbda/2) * np.dot(w.T, w)
        
    # compute gradient of l2-regulariser
    g: np.array = (lbda * w)
    return r, g

### Exercise 4

Fill in the code for the function predict(w,x) which predicts the class label $y$ for a data point $\textbf{x}$ or a matrix $X$ of data points (row-wise) for a previously trained linear model $\textbf{w}$. If there is only a data point given, the function is supposed to return a scalar value. If a matrix is given a vector of predictions is supposed to be returned.

In [23]:
def predict(w, X):
    outputs = np.dot(X,w)
    
    preds = outputs
    for value_idx in range(len(outputs)):
        if outputs[value_idx] >= 0:
            preds[value_idx] = 1
        else:
            preds[value_idx] = -1
    return preds

In [7]:
# possibility using: 2*h-1 ? - gives 1 and -1
# or np.sign

### Exercise 5

#### 5.1 
Train a linear model on the training data and classify all 187 test instances afterwards using the function predict. 
Please note that the given class labels are in the range $\left \{0,1 \right \}$, however the learning algorithm expects a label in the range of $\left \{-1,1 \right \}$. Then, compute the accuracy of your trained linear model on both the training and the test data. 

In [24]:
# Transform the class labels to be in the range of {-1, 1) instead of {0, 1}.
y_train_t = y_train * 2 - 1
y_test_t = y_test * 2 - 1


# Set the regularization parameter lambda
lbda = 0.01


# Train the linear model on the training data
w = learn_reg_ERM(X_train, y_train_t, lbda)

# Make predictions on the training and test data
y_train_pred = predict(w, X_train)
y_test_pred = predict(w, X_test)

# Calculate the accuracy on the training and test data
train_accuracy = np.mean(y_train_pred == y_train_t)
test_accuracy = np.mean (y_test_pred == y_test_t)

print(f'Training Accuracy: {round(train_accuracy*100, 2)}%')
print(f'Test Accuracy: {round(test_accuracy*100, 2)}%')

loss: 1.4607794994186016
loss: 10.991022914453243
loss: 1058.0249999999983
Training Accuracy: 50.0%
Test Accuracy: 91.98%


In [25]:
# loss only printed 3 times ?

#### 5.2
Compare the accuracy of the linear model with the accuracy of a random forest and a decision tree (non-linear models) on the training and test data set.

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Initialize classifiers
rf = RandomForestClassifier()
dt = DecisionTreeClassifier()

# Fit the classifiers
rf.fit(X_train, y_train)
dt.fit(X_train, y_train)

# Predict the labels of the training data
y_train_pred_rf = rf.predict(X_train)
y_train_pred_dt = dt.predict(X_train)

# Predict the labels of the test data 
y_test_pred_rf = rf.predict(X_test)
y_test_pred_dt = dt.predict(X_test)

# Compute the training accuracy 
train_accuracy_rf = accuracy_score(y_train, y_train_pred_rf)
train_accuracy_dt = accuracy_score(y_train, y_train_pred_dt)

# Compute the test accuracy 
test_accuracy_rf = accuracy_score(y_test, y_test_pred_rf)
test_accuracy_dt = accuracy_score(y_test, y_test_pred_dt)

# Print accuracies 
print(f'Random Forest Training Accuracy: {round(train_accuracy_rf*100, 2)}%')
print(f'Random Forest Test Accuracy: {round(test_accuracy_rf*100, 2)}%')

print(f'Dicision Tree Training Accuracy: {round(train_accuracy_dt*100, 2)}%')
print(f'Dicision Tree Test Accuracy: {round(test_accuracy_dt*100, 2)}%')

Random Forest Training Accuracy: 93.75%
Random Forest Test Accuracy: 78.07%
Dicision Tree Training Accuracy: 93.75%
Dicision Tree Test Accuracy: 67.91%
